# Pythonにおける関数呼び出し時の引数の扱い

前提として、Pythonにおける代入の挙動をおさらい。

In [6]:
a_1 = 1
print(f"{id(a_1)=}, {a_1=}")

a_1 += 1
print(f"{id(a_1)=}, {a_1=}")

id(a_1)=4376655696, a_1=1
id(a_1)=4376655728, a_1=2


Pythonで変数への再代入を行った場合、変数の指すインスタンスのID（メモリアドレス）が変わる。それを踏まえて、関数内で同様の操作を行う。

In [5]:
x_1 = 1
x_2 = 2

def add(x_1: int, x_2: int) -> int:
    print("add")
    print(f"{id(x_1)=}, {id(x_2)=}")
    x_1 = x_1 + x_2
    print("x_1 = x_1 + x_2")
    print(f"{id(x_1)=}, {id(x_2)=}")
    return x_1

print(f"{id(x_1)=}, {id(x_2)=}")
x_1 = add(x_1, x_2)
print("added")
print(f"{id(x_1)=}")

id(x_1)=4376655696, id(x_2)=4376655728
add
id(x_1)=4376655696, id(x_2)=4376655728
x_1 = x_1 + x_2
id(x_1)=4376655760, id(x_2)=4376655728
added
id(x_1)=4376655760


ここまでint型の挙動を見てきた。int型のようなプリミティブ型（違うが）の変数は、再代入によって値が更新されるから、関数内で呼び出しのメモリアドレスを持っていても意図せぬ挙動を生むことはない（メモリアドレスを直指定して値を書き換えるようなことをしない限りは）。では配列のような高水準オブジェクトではどうか。

In [10]:
ls_1 = [1]

def append_5(ls_1: list[int]) -> list[int]:
    print("append_5")
    ls_1.append(5)
    print(f"{id(ls_1)=}, {ls_1=}")
    return ls_1

print(f"{id(ls_1)=}, {ls_1=}")
ls_1 = append_5(ls_1)
print("appended")
print(f"{id(ls_1)=}, {ls_1=}")


id(ls_1)=4422151232, ls_1=[1]
append_5
id(ls_1)=4422151232, ls_1=[1, 5]
appended
id(ls_1)=4422151232, ls_1=[1, 5]


再代入を伴わない操作の場合、メソッドの内側での操作であったとしても、操作は実引数のリストに対して行われる。

## 何が問題なのか

まず、オブジェクトを可変にするのであれば、再代入によって参照するメモリアドレスが変わることで、値が期待通りに変わらないバグがあり得る。


In [19]:
def extend_with_sort(ls_1: list[int], ls_2: list[int]):
    ls_1 = ls_1 + ls_2
    ls_1.sort()

ls_1 = [1,3,5]
ls_2 = [2,4,6]
print("Before")
print(f"{ls_1=}")
extend_with_sort(ls_1, ls_2)
print("After")
print(f"{ls_1=}")  # 意図せずextendされていない


Before
ls_1=[1, 3, 5]
After
ls_1=[1, 3, 5]


逆に、再代入を許すのであれば、オブジェクトの不変性を強調しないと、呼び出し元の値が意図せず変わってしまうバグがあり得る。

In [17]:
def affine_transform_immutable(point: list[float], matrix):
    # オブジェクトに対する変更が、意図せずに伝播する
    point.append(1)

    transformed = [
        matrix[0][0] * point[0] + matrix[0][1] * point[1] + matrix[0][2] * point[2],
        matrix[1][0] * point[0] + matrix[1][1] * point[1] + matrix[1][2] * point[2],
        matrix[2][0] * point[0] + matrix[2][1] * point[1] + matrix[2][2] * point[2]
    ]

    return transformed

point = [1.0, 2.0]
scaling = [
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 1]
]

print("Before")
print(f"{point=}")
result = affine_transform_immutable(point, scaling)
print("After")
print(f"{point=}")  # 意図せず同次座標になってしまっている
print(f"{result=}")


Before
point=[1.0, 2.0]
After
point=[1.0, 2.0, 1]
result=[2.0, 4.0, 1.0]


## 考えられる対策

再代入の禁止、またはミュータブルなメソッドの禁止が考えられる。ただしリンターなどにそのような設定があるかは調べきれていない。